# Generation of Configuration Files for PhysiCell Search Grids

In [15]:
import numpy as np 
import pandas as pd
import os, sys, hashlib, itertools, math
from itertools import product
from pathlib import Path
from tqdm import tqdm

In [16]:
from sampling import LHSampler, sampling_sweep

In [17]:
print(os.getcwd())
proj_dir = os.getcwd()[0:-4]
print(proj_dir)

/home/ubuntu/projects/physim-calibration/code
/home/ubuntu/projects/physim-calibration/


In [25]:
# set output directories
run_name = "parameter_sampling_60min"
output_dir = f"output/{run_name}"

In [19]:
# helper functions
def combine(df, string_list, new_column_name='File'):
    """
    Calculates all possible combinations of rows from a DataFrame with values from a list of strings.

    Args:
        df: The input pandas DataFrame.
        string_list: A list of strings.
        new_column_name: The name of the new column to store the string values.

    Returns:
        A new DataFrame with all combinations.
    """

    # Create a new DataFrame with repeated rows for each string
    repeated_df = pd.concat([df] * len(string_list), ignore_index=True)

    # Add a new column with the string values
    repeated_df[new_column_name] = string_list * len(df) 

    return repeated_df

**Strategy**

Need to generate a search grid of parameters
1. Generate a unique number/identifier for each combination of parameters
2. Load a json (?) file as input to specify ranges of parameters
3. 


1. Make a list of dictionaries
2. Each dictionary in the list is a unique combination of parameters to test
3. Each key:value pair in the dictionary is a subelement name of a xml tag and its associated value
4. Iterate through the list
    - For each key in the dictionary, change the xml subelement tag value to the appropriate value
    - Save the xml file as a new configuration file

In [20]:
# Sample dictionary with lists as values
parameter_dictionary = {
    ## spatial patterns, simulation duration, saving parameters
    #'initial_conditions.cell_positions.filename': [file.name for file in Path(f'{proj_dir}cell_patterns').iterdir() if file.is_file()],
    'max_time': [10080], # for testing purposes
    'save.full_data.interval':[60], # for testing purposes or controlling amount of output

    ## microenvironment parameters
    #'microenvironment_setup.variable[@name=\'oxygen\'].physical_parameter_set.diffusion_coefficient': [10000, 100000, 1000000],
    #'microenvironment_setup.variable[@name=\'apoptotic debris\'].physical_parameter_set.decay_rate': [round(x, 2) for x in np.linspace(0.01, 0.1, 10)],
    #'microenvironment_setup.variable[@name=\'necrotic debris\'].physical_parameter_set.diffusion_coefficient': [10000, 100000, 1000000],
    #'microenvironment_setup.variable[@name=\'pro-inflammatory factor\'].physical_parameter_set.decay_rate': [round(x, 2) for x in np.linspace(0.01, 0.1, 10)],
    #'microenvironment_setup.variable[@name=\'anti-inflammatory factor\'].physical_parameter_set.decay_rate': [round(x, 2) for x in np.linspace(0.01, 0.1, 10)],
    
    ## tumor cell parameters
    'cell_definitions.cell_definition[@name=\'malignant_epithelial_cell\'].motility.persistence_time': [round(x, 2) for x in np.linspace(0.01, 25, 50)],
    'cell_definitions.cell_definition[@name=\'malignant_epithelial_cell\'].motility.migration_bias': [round(x, 2) for x in np.linspace(0, 1, 50)],
    'cell_definitions.cell_definition[@name=\'malignant_epithelial_cell\'].motility.speed' : [round(x, 2) for x in np.linspace(1, 15, 50)],
    #'cell_definitions.cell_definition[@name=\'malignant epithelial cell\'].mechanics.cell_cell_adhesion_strength:[1,3,5],
    #'cell_definitions.cell_definition[@name=\'malignant epithelial cell\'].mechanics.cell_cell_repulsion_strength:[10],
    #'cell_definitions.cell_definition[@name=\'malignant epithelial cell\'].mechanics.relative_maximum_adhesion_distance:[1,3,5],
    'cell_definitions.cell_definition[@name=\'malignant_epithelial_cell\'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name=\'malignant_epithelial_cell\']':[round(x, 2) for x in np.linspace(0.01, 5, 50)],

    ## M0 parameters
    'cell_definitions.cell_definition[@name=\'M0_macrophage\'].motility.persistence_time': [round(x, 2) for x in np.linspace(0.01, 25, 50)],
    'cell_definitions.cell_definition[@name=\'M0_macrophage\'].motility.migration_bias': [round(x, 2) for x in np.linspace(0, 1, 50)],
    #'cell_definitions.cell_definition[@name=\'M0_macrophage\'].motility.advanced_chemotaxis.chemotactic_sensitivities.chemotactic_sensitivity[@substrate=\'apoptotic_debris\']': [round(x, 2) for x in np.linspace(0.01, 1, 10)],
    #'cell_definitions.cell_definition[@name=\'M0_macrophage\'].mechanics.cell_cell_adhesion_strength:[1,3,5],
    #'cell_definitions.cell_definition[@name=\'M0_macrophage\'].mechanics.cell_cell_repulsion_strength:[10],
    #'cell_definitions.cell_definition[@name=\'M0_macrophage\'].mechanics.relative_maximum_adhesion_distance:[1,3,5],
    #'cell_definitions.cell_definition[@name=\'M0_macrophage\'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name=\'tumor\']:[2],
    #'cell_definitions.cell_definition[@name=\'M0_macrophage\'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name=\'hybrid\']:[1.3],

    ## M1 parameters
    'cell_definitions.cell_definition[@name=\'M1_macrophage\'].motility.persistence_time': [round(x, 2) for x in np.linspace(0.01, 25, 50)],
    'cell_definitions.cell_definition[@name=\'M1_macrophage\'].motility.migration_bias': [round(x, 2) for x in np.linspace(0, 1, 50)],
    #'cell_definitions.cell_definition[@name=\'M1_macrophage\'].motility.advanced_chemotaxis.chemotactic_sensitivities.chemotactic_sensitivity[@substrate=\'apoptotic_debris\']': [round(x, 2) for x in np.linspace(0.01, 1, 10)],
    #'cell_definitions.cell_definition[@name=\'M1_macrophage\'].mechanics.cell_cell_adhesion_strength:[1,3,5],
    #'cell_definitions.cell_definition[@name=\'M1_macrophage\'].mechanics.cell_cell_repulsion_strength:[10],
    #'cell_definitions.cell_definition[@name=\'M1_macrophage\'].mechanics.relative_maximum_adhesion_distance:[1,3,5],
    #'cell_definitions.cell_definition[@name=\'M1_macrophage\'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name=\'tumor\']:[2],
    #'cell_definitions.cell_definition[@name=\'M1_macrophage\'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name=\'hybrid\']:[1.3],
    'cell_definitions.cell_definition[@name=\'M1_macrophage\'].cell_interactions.dead_phagocytosis_rate': [round(x, 2) for x in np.linspace(0.001, 1, 50)],

    ## M2 parameters
    'cell_definitions.cell_definition[@name=\'M2_macrophage\'].motility.persistence_time': [round(x, 2) for x in np.linspace(0.01, 10, 10)],
    'cell_definitions.cell_definition[@name=\'M2_macrophage\'].motility.migration_bias': [round(x, 2) for x in np.linspace(0, 1, 50)],
    #'cell_definitions.cell_definition[@name=\'M2_macrophage\'].motility.advanced_chemotaxis.chemotactic_sensitivities.chemotactic_sensitivity[@substrate=\'apoptotic debris\']': [round(x, 2) for x in np.linspace(0.01, 1, 10)],
    #'cell_definitions.cell_definition[@name=\'M2_macrophage\'].mechanics.cell_cell_adhesion_strength:[1,3,5],
    #'cell_definitions.cell_definition[@name=\'M2_macrophage\'].mechanics.cell_cell_repulsion_strength:[10],
    #'cell_definitions.cell_definition[@name=\'M2_macrophage\'].mechanics.relative_maximum_adhesion_distance:[1,3,5],
    #'cell_definitions.cell_definition[@name=\'M2_macrophage\'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name=\'tumor\']:[2],
    #'cell_definitions.cell_definition[@name=\'M2_macrophage\'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name=\'hybrid\']:[1.3],
    'cell_definitions.cell_definition[@name=\'M2_macrophage\'].cell_interactions.dead_phagocytosis_rate': [round(x, 2) for x in np.linspace(0.001, 1, 50)],

    ## effector T cell parameters
    # hypothesis: higher T cell attack rate will facilitate tumor killing before exhaustion can develop
    'cell_definitions.cell_definition[@name=\'effector_T_cell\'].cell_interactions.attack_rates.attack_rate[@name=\'malignant_epithelial_cell\']': [round(x, 2) for x in np.linspace(0.05, 5, 50)],
    
    ## exhausted T cell parameters
    # exhausted T cells are hypoactive, but can still secrete cytokines. 
    # hypothesis: stimulating the exhausted T cell population to secrete pro-inflammatory factor will stimulate tumor
    # cell killing
    'cell_definitions.cell_definition[@name=\'exhausted_T_cell\'].secretion.substrate[@name=\'pro-inflammatory_factor\'].secretion_rate': [round(x, 2) for x in np.linspace(0.001, 1.0, 50)],
    'cell_definitions.cell_definition[@name=\'exhausted_T_cell\'].secretion.substrate[@name=\'pro-inflammatory_factor\'].secretion_target': [round(x, 2) for x in np.linspace(0.001, 1.0, 50)]

}

In [26]:
lhs = LHSampler(parameter_dictionary, seed=123)
lhs.sample(25, verbose = False, optimization='random-cd')
samples_df = lhs.get_samples_df()
samples_df.head()

,max_time,save.full_data.interval,cell_definitions.cell_definition[@name='malignant_epithelial_cell'].motility.persistence_time,cell_definitions.cell_definition[@name='malignant_epithelial_cell'].motility.migration_bias,cell_definitions.cell_definition[@name='malignant_epithelial_cell'].motility.speed,cell_definitions.cell_definition[@name='malignant_epithelial_cell'].mechanics.cell_adhesion_affinities.cell_adhesion_affinity[@name='malignant_epithelial_cell'],cell_definitions.cell_definition[@name='M0_macrophage'].motility.persistence_time,cell_definitions.cell_definition[@name='M0_macrophage'].motility.migration_bias,cell_definitions.cell_definition[@name='M1_macrophage'].motility.persistence_time,cell_definitions.cell_definition[@name='M1_macrophage'].motility.migration_bias,cell_definitions.cell_definition[@name='M1_macrophage'].cell_interactions.dead_phagocytosis_rate,cell_definitions.cell_definition[@name='M2_macrophage'].motility.persistence_time,cell_definitions.cell_definition[@name='M2_macrophage'].motility.migration_bias,cell_definitions.cell_definition[@name='M2_macrophage'].cell_interactions.dead_phagocytosis_rate,cell_definitions.cell_definition[@name='effector_T_cell'].cell_interactions.attack_rates.attack_rate[@name='malignant_epithelial_cell'],cell_definitions.cell_definition[@name='exhausted_T_cell'].secretion.substrate[@name='pro-inflammatory_factor'].secretion_rate,cell_definitions.cell_definition[@name='exhausted_T_cell'].secretion.substrate[@name='pro-inflammatory_factor'].secretion_target
0,10080.0,60.0,11.519753,0.981354,5.941493,0.821968,20.385355,0.508937,14.482275,0.924404,0.410587,4.307712,0.039421,0.024154,4.810020,0.014802,0.522904
1,10080.0,60.0,21.483242,0.676017,10.040252,3.902641,8.423908,0.885278,20.377831,0.798105,0.462528,6.748459,0.934979,0.323315,0.274791,0.271274,0.570014
2,10080.0,60.0,24.489955,0.749402,14.771189,0.304002,23.664160,0.296670,21.763389,0.108273,0.036189,5.064178,0.436025,0.457216,3.119653,0.556985,0.418381
3,10080.0,60.0,9.078891,0.334380,7.603146,2.841902,19.540434,0.587049,18.637107,0.307809,0.131543,0.762017,0.862502,0.991449,4.203597,0.968505,0.627826
4,10080.0,60.0,6.053159,0.859458,12.148998,3.686872,7.936477,0.180465,10.599816,0.068314,0.299481,3.485400,0.919764,0.411699,1.332178,0.885064,0.855058


In [23]:
# adding ics perturbations with directory to ics files
ics_replicates = [file.name for file in Path(f'{proj_dir}cell_patterns').iterdir() if file.is_file()]
samples_df = combine(samples_df, ics_replicates, new_column_name='initial_conditions.cell_positions.filename')
samples_df['initial_conditions.cell_positions.folder'] = '../cell_patterns'
samples_df['save.SVG.enable'] = 'false'

In [54]:
## OLD - Switch to parameter sampling method
# Generate all possible combinations
# combinations = list(product(*data.values()))
# all_keys = data.keys()

In [24]:
# set a naming variable for the parameter run
PARAM_RUN_NAME = 5

## Parameter Sampling
all_keys = list(samples_df.columns)
run_ids = [] # to collect the unique identifier codes for each run

with open(f"{proj_dir}/parameter_runs/parameter_sets_{PARAM_RUN_NAME}.txt", "w") as f:
        f.write("#####\n")
        f.write("# <key> <value> pairs, where <key> is the first unique node name found in the xml.\n")
        for index, row in tqdm(samples_df.iterrows()):
            # generate a unique identifier
            #row_str = ''.join(map(str, row)).encode('utf-8')
            #run_id = hashlib.md5(str(row_str).encode('utf-8')).hexdigest() # calculates the md5 hash of the combination and 
            #id = f""
            run_id = str(index).zfill(math.floor(math.log(samples_df.shape[0], 10))+1)
            f.write(f"folder {output_dir}/sim-{PARAM_RUN_NAME}-{run_id}\n")
            for i, k in enumerate(all_keys):
                v = row.iloc[i]
                f.write(f"{k} {v}\n")
            f.write(f"run_it {run_id}\n")
            run_ids.append(run_id)
#samples_df['run_identifier'] = run_ids
samples_df.to_csv(f"{proj_dir}/parameter_runs/parameter_sets_map_{PARAM_RUN_NAME}.csv")

75it [00:00, 5832.01it/s]
